# Lecture 4 - Bootstrap testing on the CCHS dataset CART model


## Retrieve the data from the course package

### The Canadian Community Health Survey CCHS was designed to gather health-related data at the sub-provincial levels of geography (health region or combined health regions). 

### **Please note:** data provided here is a synthetic version designed specifically for the educational purpose of this course.

In [8]:
library(magrittr)

df <- epi7913A::cchs %>% dplyr::slice_sample(prop=0.1)
head(df)

,age,sex,CANHEARTbin,householdsize,education,maritalstatus,immigration,houseincome
,<fct>,<fct>,<int>,<fct>,<fct>,<fct>,<int>,<fct>
1,5,1,0,2,2,3,0,1
2,4,1,1,1,2,1,0,3
3,7,2,1,2,1,3,0,2
4,5,2,1,2,4,2,0,2
5,6,2,0,2,2,3,0,2
6,4,2,0,1,1,1,0,3


## The following R code, implements a bootstrap sampling method to test a CART model

### - Remove incomplete rows (containing NAs) and set the random seed.

In [9]:
df <- df[complete.cases(df),] #remove rows with NAs
set.seed(17)  # set a random seed

### - Define a function that builds a decision tree  (CART) model with default hyperparameters

In [10]:
# function build a tree model with default hyperparameters
buildTree <- function(train) 
{
    tree_model <- rpart::rpart(CANHEARTbin ~ ., data = train, method = "class")
    return(tree_model)
}

### - Define a function to evaluate the tree (CART) model on test data

In [11]:
# function to evaluate tree model on test data
evalTree <- function(tree_model, train, test)
{
  tree_predictions <- tryCatch(error=function(cnd) {print(cnd); cbind(rep(NA, 10), rep(NA, 10)) }, 
                               { predict(tree_model, test, type = 'prob') } )

  if (sum(is.na(tree_predictions[,2]))==0)
      r1 <- MLmetrics::LogLoss(tree_predictions[,2], as.numeric(as.character(train$CANHEARTbin)))
  else
      r1 <- NA
  
  return(r1)
} 

## Now we are going to estimate the error using a bootstrap method of __Harrell et al. (https://www4.stat.ncsu.edu/~lu/ST745/sim_modelchecking.pdf)__. 

### **Note:** we are dealing with logloss, therefore, the calculations are adjusted from the original paper (as opposed to a performance metric).

### - First, compute the error on the original data

In [12]:
# first compute the error on the original data
orig.tree <- buildTree(df)
e.orig <- evalTree(orig.tree, df, df)
cat("\n","Original error:", e.orig, "\n")


 Original error: 0.6231971 


### - Then compute the bootstrap optimism

In [13]:
# then compute the bootstrap optimism
opt <- mean(sapply(seq(100), function(i) {
    boot.sample<-df[sample(nrow(df), replace=T),]
    boot.model<-buildTree(boot.sample)
    e.orig.i<-evalTree(boot.model, boot.sample, df)
    e.boot.i<-evalTree(boot.model, boot.sample, boot.sample)
    return(e.orig.i - e.boot.i) # since it is an error reverse
    }), na.rm=T)

cat("\nOptimisim:", opt, "\n")


Optimisim: 0.2029782 


### - Then add bootstrap optimism to original error

In [14]:
cat("\nOverall error:", e.orig + opt , "\n")


Overall error: 0.8261753 


### There are other bootstrapping approaches that can be used to compute the error but in general this approach provides quite a robust result.